In [1]:
import nltk
from nltk import word_tokenize
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('tagsets')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Unzipping help/tagsets.zip.


True

In [2]:
nltk.help.upenn_tagset()

$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or

In [4]:
sentence=word_tokenize("Third wave of corons virus is here")
nltk.pos_tag(sentence)

[('Third', 'JJ'),
 ('wave', 'NN'),
 ('of', 'IN'),
 ('corons', 'NNS'),
 ('virus', 'NN'),
 ('is', 'VBZ'),
 ('here', 'RB')]

In [5]:
import numpy as np
import pandas as pd
from sklearn import model_selection,naive_bayes,svm

In [8]:
df=pd.read_csv("/content/Reviews.csv",engine='python',error_bad_lines=False)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
Skipping line 22006: unexpected end of data


In [9]:
df.head()


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [11]:
df.shape


(22004, 10)

In [13]:
df.isnull().sum()

Id                        0
ProductId                 0
UserId                    0
ProfileName               0
HelpfulnessNumerator      0
HelpfulnessDenominator    0
Score                     0
Time                      0
Summary                   0
Text                      0
dtype: int64

In [15]:
df['Score'].value_counts()

5    13844
4     3142
1     1977
3     1829
2     1212
Name: Score, dtype: int64

In [17]:
df['postive rtings']=np.where(df['Score']>=3,1,0)


In [18]:
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,postive rtings
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,1
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,0
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,1
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,0
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...,1


In [19]:
df['postive rtings'].value_counts()

1    18815
0     3189
Name: postive rtings, dtype: int64

In [20]:
from sklearn.model_selection import train_test_split

In [22]:
x_train,x_test,y_train,y_test=train_test_split(df["Text"],df["postive rtings"],random_state=50)

In [24]:
print(x_train)

14259    I have four dogs, two males and two females, o...
4400     What a steal!  I make a lot of Gingerbread Coo...
11595    Hypoallergenic formula was necessary for our d...
17631    This mix is so good and so versatile that you ...
8803     This walnut oil is the best on the market. It ...
                               ...                        
14565    My husband and I always a cup of Big Easy Bold...
15649    My 2 year old German Shepherd loves this treat...
10123    I have not had pork rinds in more than 20 year...
5600     Very good coffee.  Can't always find it in our...
14000    My Malamutes aren't too picky, except one who ...
Name: Text, Length: 16503, dtype: object


In [25]:
print(y_train)

14259    1
4400     1
11595    1
17631    1
8803     1
        ..
14565    1
15649    1
10123    1
5600     1
14000    1
Name: postive rtings, Length: 16503, dtype: int64


In [27]:
from sklearn.feature_extraction.text import CountVectorizer

In [32]:
vect=CountVectorizer(min_df=5,ngram_range=(1,2),).fit(x_train)

In [34]:
len(vect.get_feature_names())

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


42906

In [36]:
x_train_vectorized=vect.transform(x_train)

In [37]:
from sklearn.naive_bayes import MultinomialNB
model=MultinomialNB()
model.fit(x_train_vectorized,y_train)

MultinomialNB()

In [40]:
predictions=model.predict(vect.transform(x_test))

In [41]:
print(predictions)

[1 1 1 ... 1 1 1]


In [42]:
from sklearn.metrics import classification_report,confusion_matrix

In [43]:
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))

[[ 570  262]
 [ 273 4396]]
              precision    recall  f1-score   support

           0       0.68      0.69      0.68       832
           1       0.94      0.94      0.94      4669

    accuracy                           0.90      5501
   macro avg       0.81      0.81      0.81      5501
weighted avg       0.90      0.90      0.90      5501



In [44]:
from sklearn.metrics import roc_auc_score
print("AUC Score is",roc_auc_score(y_test,predictions))

AUC Score is 0.8133126946142313
